In [1]:
from math import log, sqrt, pi, exp, isclose
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq
from datetime import datetime, date
import numpy as np
from yahoo_fin import options
from yahoo_fin import stock_info as si

In [2]:
## Black-Scholes Formulas ##
def d1(S, K, t, r, q, vola):
    return(log(S/K)+(r-q+vola**2/2.)*t)/(vola*sqrt(t))

def d2(S, K, t, r, q, vola):
    return d1(S, K, t, r, q, vola)-vola*sqrt(t)

def bs_call(S, K, t, r, q, vola):
    return S*exp(-q*t)*norm.cdf(d1(S, K, t, r, q, vola))-K*exp(-r*t)*norm.cdf(d2(S, K, t, r, q, vola))

def bs_put(S, K, t, r, q, vola):
    return K*exp(-r*t)-S*exp(-q*t)+bs_call(S, K, t ,r, q, vola)

In [3]:
## Implied volatility calculation formulas ##

def call_implied_volatility(S, K, t, r, q, call_market_price, a=-2.0, b=2.0, xtol=1e-6):
    _S, _K, _t, _r, _q, _call_market_price = S, K, t, r, q, call_market_price
    
    # define a nested function with only volatility as input
    def call_iv_objective_func(vola):
        return _call_market_price - bs_call(_S, _K, _t, _r, _q, vola)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(call_iv_objective_func, a=a, b=b, xtol=xtol)
        
        # if the results are too small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    except ValueError:
        return np.nan
    
def put_implied_volatility(S, K, t, r, q, put_market_price, a=-2.0, b=2.0, xtol=1e-6):
    _S, _K, _t, _r, _q, _put_market_price = S, K, t, r, q, put_market_price
    
    # define a nested function with only volatility as input
    def put_iv_objective_func(vola):
        return _put_market_price - bs_put(_S, _K, _t, _r, _q, vola)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(put_iv_objective_func, a=a, b=b, xtol=xtol)
        
        # if the results are too small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    except ValueError:
        return np.nan

#Test functions
S = 45.0
K = 45.0
t = 164.0/365.0
r = 0.02
q = 0.014
vola = 0.25

call_price = bs_call(S, K, t, r, q, vola)
put_price = bs_put(S, K, t, r, q, vola)
print('Make sure that '+ str(call_implied_volatility(S, K, t, r, q, call_price)) + ' is close to 0.25')
print('Make sure that '+ str(put_implied_volatility(S, K, t, r, q, put_price)) + ' is close to 0.25')

Make sure that 0.24999985507818948 is close to 0.25
Make sure that 0.24999985507818948 is close to 0.25


In [4]:
## Market data fetch ##

ticker = "spy"
expiration_date = datetime(2021, 9, 17)

chain = options.get_options_chain(ticker, expiration_date)
call_chain = chain["calls"]
put_chain = chain["puts"]
options.get_expiration_dates(ticker);

In [5]:
pd.set_option('display.max_rows', 500) #Show n Dataframe rows

In [6]:
## Build data table for IV computation ##

call_mid_price = (call_chain["Ask"] - call_chain["Bid"])/2. + call_chain["Bid"]
put_mid_price = (put_chain["Ask"] - put_chain["Bid"])/2. + put_chain["Bid"]
time_to_expiration = float(np.busday_count(datetime.now().date(), expiration_date.date())) / 252.
underlying_price = si.get_live_price(ticker)

#Tweak those two for ATM Vols to match at the end
risk_free_rate = 0.009
dividend_yield = 0.013759

data = {'CallStrike': call_chain["Strike"],
        'CallPrice': call_mid_price,
        'PutStrike': put_chain["Strike"],
        'PutPrice' : put_mid_price,
        'TimeToExp' : time_to_expiration,
        'UnderlyingPrice' : underlying_price,
        'RiskFreeRate' : risk_free_rate,
        'DividendYield' : dividend_yield}
data_chain = pd.DataFrame(data)
print(data_chain)

     CallStrike  CallPrice  PutStrike  PutPrice  TimeToExp  UnderlyingPrice  \
0         115.0    280.080      115.0     0.005   0.063492           447.25   
1         120.0    302.060      120.0     0.005   0.063492           447.25   
2         130.0    300.990      125.0     0.005   0.063492           447.25   
3         135.0    295.995      130.0     0.005   0.063492           447.25   
4         140.0    307.755      135.0     0.005   0.063492           447.25   
5         145.0    286.000      140.0     0.005   0.063492           447.25   
6         150.0    175.955      145.0     0.005   0.063492           447.25   
7         155.0    208.665      150.0     0.005   0.063492           447.25   
8         170.0    205.125      155.0     0.005   0.063492           447.25   
9         175.0    255.930      160.0     0.005   0.063492           447.25   
10        180.0    237.090      165.0     0.005   0.063492           447.25   
11        185.0    190.300      170.0     0.005   0.

In [7]:
## Compute implied volatility chains ##
def _get_call_implied_volatility(series):
    S = series['UnderlyingPrice']
    K = series['CallStrike']
    t = series['TimeToExp']
    r = series['RiskFreeRate']
    q = series['DividendYield']
    call_price = series['CallPrice']
    
    return float(globals().get('call_implied_volatility')(S, K, t, r, q, call_price))

def _get_put_implied_volatility(series):
    S = series['UnderlyingPrice']
    K = series['PutStrike']
    t = series['TimeToExp']
    r = series['RiskFreeRate']
    q = series['DividendYield']
    put_price = series['PutPrice']
    
    return float(globals().get('put_implied_volatility')(S, K, t, r, q, put_price))

data_chain['CallImpliedVol'] = data_chain.apply(_get_call_implied_volatility, axis=1)

data_chain['PutImpliedVol'] = data_chain.apply(_get_put_implied_volatility, axis=1)

call_data = {'Strike' : data_chain['CallStrike'],
        'CallImpliedVol' : data_chain['CallImpliedVol']}
put_data = {'Strike' : data_chain['PutStrike'],
        'PutImpliedVol' : data_chain['PutImpliedVol']}

call_vol_chain = pd.DataFrame(call_data)
put_vol_chain = pd.DataFrame(put_data)
call_vol_chain.dropna(inplace=True)
put_vol_chain.dropna(inplace=True)

E:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
E:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
E:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


In [8]:
#Group both chains in one


vol_chain = call_vol_chain.copy(deep=True)
vol_chain['PutImpliedVol'] = 0.

for i, row in call_vol_chain.iterrows():
    vol_chain.loc[i, 'PutImpliedVol'] = next(iter(put_vol_chain.loc[put_vol_chain['Strike'] == row['Strike']]['PutImpliedVol']), np.nan)

vol_chain.dropna(inplace=True)

In [9]:
#Show computed IVs
print(vol_chain)

     Strike  CallImpliedVol  PutImpliedVol
24    250.0        1.141062       0.770011
25    255.0        1.096477       0.745361
26    260.0        1.032867       0.721165
28    270.0        0.962163       0.694742
30    280.0        0.944249       0.663573
31    285.0        0.879474       0.653035
32    290.0        0.843403       0.629728
33    295.0        0.825960       0.617205
36    300.0        0.787008       0.594266
37    301.0        0.946247       0.598660
40    305.0        0.767641       0.588260
45    310.0        0.756399       0.568994
50    315.0        0.706340       0.552855
55    320.0        0.678206       0.533255
56    321.0        0.674398       0.534372
58    323.0        0.665854       0.525325
60    325.0        0.652172       0.516323
61    326.0        0.675852       0.516934
62    327.0        0.641993       0.512424
65    330.0        0.652485       0.503634
68    333.0        0.618723       0.494502
70    335.0        0.595936       0.489582
71    336.0